
# Member Survey
#### surveys@aitec.org
#### April 26, 2017


In [ ]:
## import libs
import re
import numpy as np
import pandas as pd

# offline plotly
# https://plot.ly/python/offline/
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode()

# functions to draw plotly charts in a standard way
from aitec_plotly import *
# functions to pull data from survey data and call aitec_plotly functions
from aitec_convenience import *

# include a YouTube on last slide
from IPython.display import YouTubeVideo


In [ ]:
## load CSVs

rootdir = "C:/Users/druce/R/aitec/python"
datadir = "C:/Users/druce/R/aitec/Data_All_170414/CSV"
csv_prefix = "Sheet_"
csvfile = "chartdata.csv"
buysidefile = "buyside.csv"
countfile = "count_user_answers.csv"
min_answers=10

def do_merge(newfile, dataframe):
    tmpframe = pd.read_csv(newfile, sep=",")
    delcols = ["CollectorID","StartDate","EndDate","IP Address","Email Address","First Name","LastName","Custom Data",]
    
    tmpframe.drop(delcols, inplace=True,axis=1)
    tmpframe.set_index(['RespondentID'], drop=True, append=False, inplace=True, verify_integrity=True)
    
    retframe = pd.concat([chartdata, tmpframe], axis=1, join_axes=[chartdata.index])    
    #retframe = chartdata.merge(tmpframe, right_on='RespondentID', left_on='RespondentID', how='inner')
    return(retframe)

def import_raw():
    """import raw data and save to chartdata.csv"""
    global chartdata
    i=1
    nextfile = "%s/%s%d.csv" % (datadir, csv_prefix, i)
    chartdata = pd.read_csv(nextfile, sep=",")
    delcols = ["CollectorID","StartDate","EndDate","IP Address","Email Address","First Name","LastName","Custom Data",
               "What is the name of your firm? (please confirm) - Open-Ended Response"]
    
    chartdata.drop(delcols, inplace=True,axis=1)
    chartdata.set_index(['RespondentID'], drop=True, append=False, inplace=True, verify_integrity=True)

    print("processed CSV %s" % nextfile)
    print(chartdata.shape)
    #print(chartdata[[0]])

    while True:
        i += 1
        nextfile = "%s/%s%d.csv" % (datadir, csv_prefix, i)
        try:
            chartdata = do_merge(nextfile, chartdata)
            print("processed CSV %s" % nextfile)
        except IOError:
            # print("%s not found" % nextfile)
            break
    print chartdata.shape

    # insert placeholder columns (did first version with extra columns and need column numbers to line up)
    for i in range(6):
        chartdata.insert(0, "%s" % i, i)    
    print("Saving %s/%s" % (datadir, csvfile))
    chartdata.to_csv("%s/%s" % (datadir, csvfile))

# only do this once to merge/scrub raw SurveyMonkey files
#import_raw()

# thereafter read chartdata.csv from root directory
chartdata = pd.read_csv("%s/%s" % (datadir, csvfile), sep=",")

# filter rows with < min_answers responses
#count how many answers each respondent completed, save to file
count_answers = pd.DataFrame(chartdata.count(axis=1))
#count_answers.sort_values([0], axis=0, ascending=True, inplace=True)    
count_answers[0]=count_answers[0]-min(count_answers[0])
count_out = pd.DataFrame(chartdata['RespondentID'])
count_out = count_out.assign(Answered=count_answers[0])        
count_out.to_csv("%s/%s" % (datadir, countfile))    
count_answers = count_answers.loc[count_answers[0] >=min_answers]
chartdata = chartdata[chartdata.index.isin(count_answers.index.values)]

# filter buy side only
question_col = 9
question_name = chartdata.columns[question_col]
#print (question_name)
chartdata = chartdata.loc[chartdata[question_name] == "Buy Side"]
chartdata.to_csv("%s/%s" % (datadir, buysidefile))
#print(len(chartdata))
#chartdata




# General Information

In [ ]:
question_col = 7
chartdict = twocolumn_barchart(chartdata, question_col, y_label="Count")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_col = 9
chartdict = onecolumn_barchart(chartdata, question_col, y_label="Count")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_col = 10
chartdict = onecolumn_barchart(chartdata, question_col, y_label="Count")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_col = 11
chartdict = onecolumn_barchart(chartdata, question_col, y_label="Count",
                              col_order=["<1bn", "1-3bn", "3-5bn", "5-10bn", "10bn-20bn", ">20bn", "No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol = 12
ncols=14
chartdict = multicolumn_barchart(chartdata, startcol, ncols,
                                 y_label="Count",
                                 othertext=True,
                                 title="Which investment strategy(s) does your firm employ?",
                                 x_label_size=10, 
                                 bottom_margin=120                                 
                                )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_col = 26
chartdict = onecolumn_barchart(chartdata, 
                               26, 
                               x_label="",
                               y_label="Count",
                               title="If you employ systematic and fundamental strategies,<br />do you segregate the two?",
                              col_order=["Yes","No","No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_cols = [27,28,29]
colnames = chartdata.columns[question_cols]
question_name = "If Yes, how do you segregate fundamental and systematic?"

chartframe = chartdata[colnames]
colnames = ["Physical office locations","Organization/teams","IT resources"]
chartframe.columns = colnames
chartframe = chartframe.fillna(value="No Answer")

#pivot    
meltframe = pd.melt(chartframe, value_vars=colnames)
pivotframe = meltframe.pivot(columns='value', values='variable')
pivotnames = ["Yes", "No", "No Answer"]
pivotframe.columns = pivotnames

tracedict = {}
for colname in colnames:
    countSeries = chartframe[colname].value_counts(sort=True, ascending=False,dropna=True)
    tracedict[colname] = countSeries

# new version use dataframe instead of dict    
chartframe = pd.DataFrame(tracedict)
# put x-axis on the index, make the columns bars in a group
chartframe = chartframe.transpose()
# fix order, skip "No Answer"
chartframe = chartframe[["Yes", "No"]]

#print(chartframe)
chartdict = aitec_bargroup(chartframe, question_name, "", "Count")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
# grid is complicated to clean up
# first get all values in one frame
# then create 4 Series, one for None, Low, Medium, High

question_cols = range(30,106)
colnames = chartdata.columns[question_cols]
question_name = "How much volume does your firm trade of each product type?"

chartframe = chartdata[colnames]
# melt multiple columns into 1, i.e. gather/unflatten
#colnums = ["%" % i for i in question_cols]
variables = [mystr[len("How much volume does your firm trade of each product type? - "):] for mystr in colnames]
chartframe.columns= variables
meltframe = pd.melt(chartframe, value_vars=variables)
meltframe = meltframe.dropna(axis=0)

# clean up values so they fit on a chart
abbrev_dict = {
    u'Governments\xa0Bonds\xa0'                     : 'Govt bonds',
    'Corporate Bonds'                               : 'Corp bonds',
    'Bank Debt / Private Debt / Direct Lending'     : 'Direct Debt',
    'MBS & CMOs'                                    : 'MBS / CMOs',
    'ABS'                                           : 'ABS',
    u'Common / Preferred\xa0Equities'               : 'Stocks',
    'Mutual Funds & ETFs'                           : 'Mut Fund/ETF',
    'MLPs'                                          : 'MLPs',
    'Real Estate & REITS'                           : 'RE/REITs',
    'Private Equity'                                : 'PE',
    'Equity swaps'                                  : 'Equity Swaps',
    'Commodity swaps & derivatives'                 : 'Commod Deriv',
    'Futures'                                       : 'Futures',
    'Credit swaps'                                  : 'Credit Swaps',
    'Currency swaps'                                : 'FX Swaps',
    'Rate swaps'                                    : 'Rate Swaps',
    'Warrants & Options'                            : 'Options',
    'F/X currency'                                  : 'FX',
    'Other'                                         : 'Other',
 } 

# count occurrences of each response
countseries = meltframe['variable'].value_counts(sort=True, ascending=False,dropna=True)
countframe = pd.DataFrame(countseries)

# split the name ie "product - high/med/low/none'
countframe["product"]=countframe.index.format()
countframe["product"]=[mystr[0:mystr.find(" - ")] for mystr in countframe["product"]]
countframe["volume"]=countframe.index.format()
countframe["volume"]=[mystr[mystr.find(" - ")+3:] for mystr in countframe["volume"]]

countframe.loc[countframe["volume"] == "Low (Occasional trades)", "volume"] = "Low"
countframe.loc[countframe["volume"] == "Medium (Active trading)", "volume"] = "Medium"
countframe.loc[countframe["volume"] == "High (Primary trading focus)", "volume"] = "High"
countframe['index'] = range(countframe.shape[0])
countframe = countframe.set_index(keys='index')
countframe.columns = ['count', 'product', 'volume']

for source, dest in abbrev_dict.items():
    # stupidly inefficient
    countframe = countframe.replace(to_replace=source, value=dest)

   
#print(list(countframe["product"]))
#print(countframe)
# split into 4 dataframes, get products as dicts
none_frame = countframe.loc[countframe["volume"] == "None"]
none_frame = none_frame.set_index(keys='product')
none_frame = pd.DataFrame(none_frame["count"])
none_frame.columns = ['None']
low_frame = countframe.loc[countframe["volume"] == "Low"]
low_frame = low_frame.set_index(keys='product')
low_frame = pd.DataFrame(low_frame["count"])
low_frame.columns = ['Low']
medium_frame = countframe.loc[countframe["volume"] == "Medium"]
medium_frame = medium_frame.set_index(keys='product')
medium_frame = pd.DataFrame(medium_frame["count"])
medium_frame.columns = ['Medium']
high_frame = countframe.loc[countframe["volume"] == "High"]
high_frame = high_frame.set_index(keys='product')
high_frame = pd.DataFrame(high_frame["count"])
high_frame.columns = ['High']

all_frame = high_frame
all_frame = pd.concat([all_frame, medium_frame], axis=1, join='outer')    
all_frame = pd.concat([all_frame, low_frame], axis=1, join='outer')    
all_frame = all_frame.fillna(value=0)

total_frame=pd.DataFrame(all_frame["High"] + all_frame["Medium"] + all_frame["Low"])
total_frame = total_frame.sort_values(0, ascending=False)
#use this order
col_order = total_frame.index.values
order_dict = dict(zip(col_order, range(len(col_order))))
all_frame["order"] = [order_dict[v] for v in all_frame.index.values]
all_frame = all_frame.sort_values("order")
del all_frame["order"]

chartdict = aitec_barstack(all_frame, question_name, "", "Count")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata, 
                                 107, 
                                 7, 
                                 y_label="Count", 
                                 title="In addition to hedge funds, do you manage…?")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_col = 114
chartdict = onecolumn_barchart(chartdata, 
                               question_col, 
                               x_label="",
                               y_label="Count",
                               col_order = ['<10', '11-30', '31-50', '51-100', '101-200', '201-300', '301-500', '>500', 'No Answer',]
                               #title = "If Yes, how do you segregate fundamental and systematic? <br />IT resources"
                              )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_cols = [115]
colnames = chartdata.columns[question_cols]
question_name = "What percentage of your firm's employees are in<br />the front office?"
x_label = "Percentage"
y_label = "Count"
val_list = chartdata[colnames[0]].tolist()

# jumps inexplicably from 8 to 14
chartdict = aitec_histogram(val_list, question_name, x_label, y_label, nbinsx=10)

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_cols = [117,118,119]
colnames = chartdata.columns[question_cols]
question_name = "In 2017, do you expect headcount to rise or fall?"

chartframe = chartdata[colnames]
colnames = ["Front Office","Back Office","Overall"]
chartframe.columns = colnames

chartframe = chartframe.fillna(value="No Answer")

#pivot    
meltframe = pd.melt(chartframe, value_vars=colnames)
pivotframe = meltframe.pivot(columns='value', values='variable')
pivotnames = ["Rise", "About the Same", "Fall", "No Answer"]
pivotframe.columns = pivotnames

tracedict = {}
for colname in colnames:
    countSeries = chartframe[colname].value_counts(sort=True, ascending=False,dropna=True)
    tracedict[colname] = countSeries

# new version use dataframe instead of dict    
chartframe = pd.DataFrame(tracedict)
# put x-axis on the index, make the columns bars in a group
chartframe = chartframe.transpose()

chartdict = aitec_bargroup(chartframe, question_name, "", "Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_cols = [120]
colnames = chartdata.columns[question_cols]
question_name = "What percentage of the firm's total full-time staff<br />is dedicated to the IT team?"
x_label = "Percentage"
y_label = "Count"

chartframe = chartdata[colnames]
chartframe = chartframe.replace(to_replace=".5", value="0.5")
chartframe = chartframe.replace(to_replace="3% (1 person)", value="3")
fixup = []
for z in chartframe[colnames[0]]:
    #print(z)
    if isinstance(z, float):
        fixup.append(z)
    elif isinstance(z, str):
        z = z.strip()
        z = re.sub("^\D*", "", z)
        z = re.sub("\D*$", "", z)
        fixup.append(float(z))
    #print(fixup[-1])

chartdict = aitec_histogram(fixup, question_name, x_label, y_label)

# in future ensure proper validation for all free from numeric fields 


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
tableframe = pd.DataFrame(fixup).describe()
tableframe.columns=['value']
tableframe['value'] = tableframe['value'].apply('{:,.1f}'.format)

In [ ]:
tableframe

In [ ]:
question_cols = range(121, 135)
colnames = chartdata.columns[question_cols]
question_name = "What percentage of your IT employees <br />do you have serving each role?"
y_label = "Percentage"
x_label = ""

chartframe = chartdata[colnames]
choplen = len("What percentage of your IT EMPLOYEES do you have serving each role? The total must add up to 100. -  ")

colnames = [mystr[choplen:] for mystr in colnames]
#print(colnames)
#cleanup for chart
colnames = ['Help Desk', 
            'App Support', 
            'Network Support', 
            'Server Support', 
            'Security', 
            'DBA', 
            'Development', 
            'QA', 
            'Proj Management', 
            'Business Analysis', 
            'Market Data', 
            'Risk', 
            'Quant', 
            'Other']
chartframe.columns = colnames

# remove no-answer rows, pad 0s
chartframe.columns = colnames
chartframe = chartframe.dropna(how='all')
chartframe = chartframe.fillna(value=0)

chartdict = aitec_boxplot(chartframe, question_name, x_label, y_label)

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
# an example of a table slide
# but not sure this is the best way
# maybe style via the ipython / pandas html render hooks
# or via css
tableframe = chartframe.describe().transpose()
# map to formatted strings
format_dict = {'count': '{:,.0f}',
               'mean': '{:,.1f}',
               'std': '{:,.1f}',
               'min': '{:,.0f}',
               '25%': '{:,.0f}',
               '50%': '{:,.0f}',
               '75%': '{:,.0f}',
               'max': '{:,.0f}',
              }

for key, value in format_dict.items():
    tableframe[key] = tableframe[key].apply(value.format)


In [ ]:
tableframe

In [ ]:
question_cols = [135]
colnames = chartdata.columns[question_cols]
question_name = "What percentage of the total full-time IT team<br />is composed of consultants?"
x_label = ""
y_label = "Count"
chartframe = chartdata[colnames]
#print(chartframe)

# in future ensure proper validation for all free from numeric fields 

fixup = []
for z in chartframe[colnames[0]]:
    if isinstance(z, float):
        fixup.append(z)
    elif isinstance(z, str):
        z = z.strip()
        z = re.sub("^\D*", "", z)
        z = re.sub("\D*$", "", z)
        fixup.append(float(z))

# so largest bin max is 100        
fixup = [99.99 if x == 100.0 else x for x in fixup]        
# print(max(fixup)        )
chartdict = aitec_histogram(fixup, question_name, x_label, y_label)

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
tableframe = pd.DataFrame(fixup).describe()
tableframe.columns=['value']
tableframe['value'] = tableframe['value'].apply('{:,.1f}'.format)

In [ ]:
tableframe

In [ ]:
question_cols = range(136, 150)
colnames = chartdata.columns[question_cols]
question_name = "What percentage of your IT consultants <br />do you have serving each role?"
y_label = "Percentage"
x_label = ""

chartframe = chartdata[colnames]
choplen = len("What percentage of your full-time IT consultants do you have serving each role? The total must add up to 100. Â Note:Â Include contractors working on projects who would equal that of a full time employee for a year (e.g. a firm may have a contract developer working 2000 hours on a project during the next 12 months that would equal 1 IT consultant -)")

colnames = [mystr[choplen:] for mystr in colnames]
#print(colnames)
#cleanup for chart
colnames = ['Help Desk', 
            'App Support', 
            'Network Support', 
            'Server Support', 
            'Security', 
            'DBA', 
            'Development', 
            'QA', 
            'Proj Management', 
            'Business Analysis', 
            'Market Data', 
            'Risk', 
            'Quant', 
            'Other']
# remove no-answer rows, pad 0s
chartframe.columns = colnames
chartframe = chartframe.dropna(how='all')
chartframe = chartframe.fillna(value=0)

chartdict = aitec_boxplot(chartframe, question_name, x_label, y_label)
#chartframe

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
from IPython.display import HTML, display
# table slide with HTML
# still not sure this is the best way
tableframe = chartframe.describe().transpose()
# map to formatted strings
format_dict = {'count': '{:,.0f}',
               'mean': '{:,.1f}',
               'std': '{:,.1f}',
               'min': '{:,.0f}',
               '25%': '{:,.0f}',
               '50%': '{:,.0f}',
               '75%': '{:,.0f}',
               'max': '{:,.0f}',
              }

for key, value in format_dict.items():
    tableframe[key] = tableframe[key].apply(value.format)

clabels = tableframe.columns.values
rlabels = tableframe.index.values

htmlstr = '<table><tr><td></td><td style="font-weight: bold; text-align: right">count</td><td style="font-weight: bold; text-align: right">mean</td><td style="font-weight: bold; text-align: right">std</td><td style="font-weight: bold; text-align: right">min</td><td style="font-weight: bold; text-align: right">25%</td><td style="font-weight: bold; text-align: right">50%</td><td style="font-weight: bold; text-align: right">75%</td><td style="font-weight: bold; text-align: right">max</td></tr><tr>{}</tr></table>'.format(
        '</tr><tr>'.join('<td>{}</td>'.format('</td><td style="text-align: right">'.join(['<span style="font-weight: bold">%s</span>' % row] + list(tableframe.loc[row,col] for col in clabels) )) for row in rlabels)
        )


In [ ]:
display(HTML(htmlstr))

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               150, 
                               x_label="",
                               y_label="Count",
                               title=None,
                              col_order=["Yes","No","No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               151, 
                               x_label="",
                               y_label="Count",
                               title=None,
                              col_order=["Yes","No","No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               152, 
                               x_label="",
                               y_label="Count",
                               title=None,
                              col_order=["Yes","No","No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               153, 
                               x_label="",
                               y_label="Count",
                               title=None,
                              col_order=["Yes","No","No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               154, 
                               x_label="",
                               y_label="Count",
                               title=None,
                              col_order=["Yes","No","No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               155, 
                               x_label="",
                               y_label="Count",
                               col_order=["Yes","No","No Answer"],
                               title = "Do you apply chargebacks or pass throughs<br />for your development team?")



In [ ]:


iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               156, 
                               x_label="",
                               y_label="Count",
                               col_order=["Yes","No","No Answer"],
                               title = "Do you apply chargebacks or pass throughs<br />for your infrastructure / support teams?")



In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               157, 
                               x_label="",
                               y_label="Count",
#                               col_order=["Yes","No","No Answer"],
                               title = "Do you divide support into separate help desk groups?")



In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_cols = range(158, 169)
colnames = chartdata.columns[question_cols]
question_name = "CTOs and CIOs - How do you spend your time?"
y_label = "Percentage"
#x_label = "Activity"
x_label = ""

chartframe = chartdata[colnames]
choplen = len("CTOs and CIOs - How do you spend your time? (must add up to 100%) - ")

colnames = [mystr[choplen:] for mystr in colnames]
#print(colnames)
#cleanup for chart
colnames = ['Strategy', 
            'Meetings', 
            'Infra mgmt', 
            'Dev mgmt', 
            'Vendor mgmt', 
            'Security', 
            'Hands-on infra', 
            'Hands-on dev', 
            'Hands-on security', 
            'Org/Team dev', 
            'Other']

chartframe.columns = colnames
# remove no-answer rows, pad 0s
chartframe.columns = colnames
chartframe = chartframe.dropna(how='all')
chartframe = chartframe.fillna(value=0)
chartdict = aitec_boxplot(chartframe, question_name, x_label, y_label)

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
tableframe = chartframe.describe().transpose()
# map to formatted strings
format_dict = {'count': '{:,.0f}',
               'mean': '{:,.1f}',
               'std': '{:,.1f}',
               'min': '{:,.0f}',
               '25%': '{:,.0f}',
               '50%': '{:,.0f}',
               '75%': '{:,.0f}',
               'max': '{:,.0f}',
              }

for key, value in format_dict.items():
    tableframe[key] = tableframe[key].apply(value.format)


In [ ]:
tableframe

In [ ]:
question_cols = range(169, 182)
colnames = chartdata.columns[question_cols]
question_name = "What's your biggest area of focus for 2017?"
y_label = "Percentage"
#x_label = "Activity"
x_label = ""

chartframe = chartdata[colnames]
choplen = len("What's your biggest area of focus for 2017? (must add up to 100%) -")

colnames = [mystr[choplen:] for mystr in colnames]
#print(colnames)
#cleanup for chart
colnames = ['Dev. projects', 'Storage', 'Cloud', 'Cybersecurity', 'Facilities', 'Datacenter', 'A/V', 'Telecom', 'Data Mgmt', 'Mobile', 'Innovation', 'Discret. projects', 'Other']

chartframe.columns = colnames
# remove no-answer rows, pad 0s
chartframe.columns = colnames
chartframe = chartframe.dropna(how='all')
chartframe = chartframe.fillna(value=0)
chartdict = aitec_boxplot(chartframe, question_name, x_label, y_label)

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_cols = range(182, 187)
question_name = "What technology are you most excited about?"

chartdict = multicolumn_barchart(chartdata,
                                 182, 
                                 5, 
                                 y_label="Count",
                                 othertext = False,
                                )

In [ ]:
iplot(chartdict, link_text="")

 

# Front Office Solutions

In [ ]:
chartdata.replace(to_replace="paladyne", value="Paladyne", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 360, 
                                 15, 
                                 y_label="Count",
                                 x_label_size=11, bottom_margin=120
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="ss&c", value="SS&C", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 375, 
                                 12, 
                                 y_label="Count",
                                 
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol = 387
ncols = 19
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 
                                 )



In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Charles River Systems", value="Charles River", inplace=True)

startcol = 406
ncols = 25
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 
                                 )



In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="custom built", value="Custom", inplace=True)
chartdata.replace(to_replace="internally developed", value="Custom", inplace=True)
chartdata.replace(to_replace="Internal system", value="Custom", inplace=True)
question_col = 431

chartdict = twocolumn_barchart(chartdata, question_col, y_label="Count")



In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol = 433
ncols=18

chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 title=None
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="In house developed system", value="Internally developed", inplace=True)
chartdata.replace(to_replace="internally developed", value="Internally developed", inplace=True)

startcol = 451
ncols = 14
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

# Back Office Solutions

In [ ]:
startcol = 465
ncols = 9
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Enfusion Integrata", value="Enfusion", inplace=True)


startcol = 474
ncols = 37
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Enfusion Integrata", value="Enfusion", inplace=True)
chartdata.replace(to_replace="Triana", value="Traiana", inplace=True)


startcol = 511
ncols = 14
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",                                 
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol=525
ncols=10
#chartdata.replace(to_replace="Enfusion Integrata", value="Enfusion", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol=535
ncols=15
#chartdata.replace(to_replace="Enfusion Integrata", value="Enfusion", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                )
                                

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Advent Axys", value="Advent Axys or APX", inplace=True)
chartdata.replace(to_replace="Advent APX (SQL upgrade version of Axys)", value="Advent Axys or APX", inplace=True)
chartdata.replace(to_replace="Advent APX", value="Advent Axys or APX", inplace=True)

question_col = 550
chartdict = twocolumn_barchart(chartdata, question_col, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_col = 552
chartdict = twocolumn_barchart(chartdata, question_col, 
                               title="Do you outsource middle/back office functions<br />to a managed services provider?",
                               y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol=554
ncols=10
#chartdata.replace(to_replace="Enfusion Integrata", value="Enfusion", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 title="If Yes, which firms are you using / considering?"
                                 )




In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol=564
ncols=8
#chartdata.replace(to_replace="Enfusion Integrata", value="Enfusion", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 title="If Yes, which functions are you outsourcing / considering?"
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
startcol=572
ncols=7
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 title ="If Yes, what were the drivers to determine the use of managed services?"
                                 )




In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 579, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Sunguard", value="Sungard", inplace=True)
startcol=581
ncols=21
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 title=None
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Eze", value="Eze Castle", inplace=True)
chartdata.replace(to_replace="Eze Castle / Voice", value="Eze Castle", inplace=True)
startcol=602
ncols=14
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 title=None
                                 )


In [ ]:
iplot(chartdict, link_text="")

# Software Development

In [ ]:
startcol=616
ncols=7
chartdict = multicolumn_barchart(chartdata,
                                 startcol, 
                                 ncols, 
                                 y_label="Count",
                                 title=None
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 623, y_label="Count")



In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata, 
                                 625, 
                                 11, 
                                 y_label="Count", 
                                 title="What programming languages do you use?<br />(Desktop/Server)")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata, 
                                 636, 
                                 13, 
                                 y_label="Count", 
                                 title="What statistical/quant analysis frameworks does your firm use?")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 649, 
                                 10, 
                                 y_label="Count", 
                                 title="What development platforms do you use<br />(Web front-end)?")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 659, 
                                 9, 
                                 y_label="Count",
                                 title="What development platforms do you use<br />(Web back-end)?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 668, 
                                 10, 
                                 y_label="Count",
                                 title="What development platforms do you use <br />(mobile)?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 678, 
                                 6, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 684, 
                                 9, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

# Security

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 693, 
                                 21, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               714, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:

chartdata.replace(to_replace="edgewave", value="EdgeWave", inplace=True)
chartdata.replace(to_replace="Edgewave", value="EdgeWave", inplace=True)
chartdata.replace(to_replace="zscaler", value="ZScaler", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 715, 
                                 11, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 726, 
                                 9, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 735, 
                                 5, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 740, 
                                 5, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 745, 
                                 32, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
chartdata.replace(to_replace="bespoke", value="Custom", inplace=True)
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="bespoke", value="Custom", inplace=True)
chartdata.replace(to_replace="winscp", value="WinSCP", inplace=True)
chartdata.replace(to_replace="Winscp", value="WinSCP", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 777, 
                                 8, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Mimecast secure send", value="Mimecast Secure Send", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 785, 
                                 10, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:


chartdict = multicolumn_barchart(chartdata,
                                 795, 
                                 9, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               804, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 805, 
                                 19, 
                                 y_label="Count",
                                 title="What Intrusion Detection/Prevention System (IDS/IPS) or <br />Unified Threat Management (UTM) vendor does your firm use?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 824, 
                                 7, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="OKTA", value="Okta", inplace=True)
chartdata.replace(to_replace="Okta Verify", value="Okta", inplace=True)
chartdata.replace(to_replace="Microsoft 2 factor", value="Microsoft", inplace=True)
chartdata.replace(to_replace="Gemoto (Safenet)", value="Gemalto", inplace=True)



chartdict = multicolumn_barchart(chartdata,
                                 831, 
                                 7, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="onboarding ADFS", value="ADFS", inplace=True)
chartdata.replace(to_replace="ADFS/SAML", value="ADFS", inplace=True)
chartdata.replace(to_replace="DUO Access Gateway", value="Duo", inplace=True)


chartdata["What SSO does your firm use? - Other (please specify).1"].replace(to_replace="MSFT", value="Microsoft", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 838, 
                                 8, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               846, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               847, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               848, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               849, 
                               x_label="",
                               y_label="Count",
                               title="What is your password policy?<br />Maximum number of failed attempts before lockout")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="keepass", value="KeePass", inplace=True)
chartdata.replace(to_replace="keypass", value="KeePass", inplace=True)
chartdata.replace(to_replace="PMP", value="Password Manager Pro", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 850, 
                                 11, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="elasticsearch", value="ELK", inplace=True)
chartdata.replace(to_replace="Proprietary on ELK stack", value="ELK", inplace=True)
chartdata.replace(to_replace="E-Sentire", value="eSentire", inplace=True)
chartdata.replace(to_replace="eSentire Log Sentry", value="eSentire", inplace=True)
chartdata.replace(to_replace="rapid7", value="Rapid7", inplace=True)


chartdict = multicolumn_barchart(chartdata,
                                 861, 
                                 10, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               871, 
                               x_label="",
                               y_label="Count",
                               col_order=["Yes","No","No Answer"]
                               #title=""
                              )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               872, 
                               x_label="",
                               y_label="Count",
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               873, 
                               x_label="",
                               y_label="Count",
                               col_order=['0-10','11-25','26-50','51-100','100+','No Answer',]
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               874, 
                               x_label="",
                               y_label="Count",
                               col_order=['0-5','6-10','11-15','16-25','25+','No Answer',]
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 875, 
                                 10, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 885, 
                                 12, 
                                 y_label="Count",
                                 title="What type of vendors are critical<br />as they host or have access to sensitive information?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               897, 
                               x_label="",
                               y_label="Count",
                               title="Are you actively reviewing vendor relationships<br />to reduce the  number of relationships?",
                               col_order=["Yes","No","No Answer"]
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 898, 
                                 10, 
                                 y_label="Count",
                                 #title=""
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 908, 
                                 21, 
                                 y_label="Count",
                                 title="What company has your firm used for penetration tests<br />and other information security assessments?",
                                 x_label_size=11,
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               929, 
                               x_label="",
                               y_label="Count",
                               #title="Are you actively reviewing vendor relationships<br />to reduce the  number of relationships?"
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="ACA APonix", value="ACA Aponix", inplace=True)
chartdata.replace(to_replace="Aponix", value="ACA Aponix", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 930, 
                                 8, 
                                 y_label="Count",
                                 #title="What company has your firm used for penetration tests<br />and other information security assessments?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               938, 
                               x_label="",
                               y_label="Count",
                               #title="Are you actively reviewing vendor relationships<br />to reduce the  number of relationships?"
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               939, 
                               x_label="",
                               y_label="Count",
                               title="If Yes, have you engaged insurance recovery attorneys<br />to review the policy language?"
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Never hired one", value="None", inplace=True)
chartdata.replace(to_replace="knowbe4", value="KnowBe4", inplace=True)
chartdata.replace(to_replace="KnowB4", value="KnowBe4", inplace=True)
chartdata.replace(to_replace="Knowbe4", value="KnowBe4", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 940, 
                                 15, 
                                 y_label="Count",
                                 title="What firms have you used to conduct<br />Cyber Security best practice training?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               955, 
                               x_label="",
                               y_label="Count",
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               956, 
                               x_label="",
                               y_label="Count",
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               957, 
                               x_label="",
                               y_label="Count",
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
#chartdata.replace(to_replace="In-house developed with outside counsel", value="Internal", inplace=True)
chartdata.replace(to_replace="esentire", value="eSentire", inplace=True)
chartdata.replace(to_replace="ESentire", value="eSentire", inplace=True)
chartdata.replace(to_replace="Esentire", value="eSentire", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 958, 
                                 11, 
                                 y_label="Count",
                                 title="What companies has your firm used to help construct<br />a BCP/Cybersecurity Incident Response Plan?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="EverBridge", value="Everbridge", inplace=True)
chartdata.replace(to_replace="VMware AirWatch", value="AirWatch", inplace=True)
chartdata.replace(to_replace="AirWatch MDM", value="AirWatch", inplace=True)
chartdata.replace(to_replace="Airwatch", value="AirWatch", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 969, 
                                 12, 
                                 y_label="Count",
                                 #title="What companies has your firm used to help construct<br />a BCP/Cybersecurity Incident Response Plan?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 981, 
                                 16, 
                                 y_label="Count",
                                 #title="What companies has your firm used to help construct<br />a BCP/Cybersecurity Incident Response Plan?"
                                )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 997, 
                                 8, 
                                 y_label="Count",
                                 title='What is your policy on "burner device" laptops for overseas travel<br />to high-risk countries?',
                                 othertext=False
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1005, 
                                 9, 
                                 y_label="Count",
                                 title='What is your policy on "burner device" handheld devices<br />for overseas travel to high-risk countries?',
                                 othertext=False
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1014, 
                                 23, 
                                 y_label="Count",
                                 title=None
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1037, 
                               x_label="",
                               y_label="Count",
                               col_order=["Yes","No","No Answer"]                               
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1038, 
                               x_label="",
                               y_label="Count",
                               col_order=["Yes","No","No Answer"]                               
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1039, 
                               x_label="",
                               y_label="Count",
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1040, 
                               x_label="",
                               y_label="Count",
                               col_order=["Yes","No","No Answer"]                               
                               #title=""
                              )

In [ ]:
iplot(chartdict, link_text="")

# Infrastructure & Data Center

In [ ]:
chartdict = default_boxplot(chartdata, 
                            1041, 
                            2, 
                            "", 
                            "Percentage")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = default_boxplot(chartdata, 
                            1043, 
                            4, 
                            "", 
                            "Percentage")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="by end of year about 10%", value="10", inplace=True)

chartdict = onecolumn_barchart(chartdata, 
                               1047, 
                               x_label="",
                               y_label="Percentage",
                               title="If applicable, what percentage of your virtual servers are in the cloud<br />(either private or public cloud)?"
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = default_boxplot(chartdata, 
                            1048, 
                            5, 
                            "", 
                            "Percentage",
                           title="What percentage of your servers are in each of these groups?")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = default_boxplot(chartdata, 
                            1053, 
                            6, 
                            "", 
                            "Percentage",
                           title="Percentage of VM's by vendor your firm has in use?"
                           )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1059, 
                                 7, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1066, 
                                 12, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1078, 
                                 21, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1099, 
                                 21, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1120, 
                                 13, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1133, 
                                 7, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="0 currently onboarding", value="0", inplace=True)

chartdict = onecolumn_barchart(chartdata, 
                               1047, 
                               x_label="",
                               y_label="Percentage",
                               title="What Percentage of your end-user workstations are VDI?"
                              )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1141, 
                                 11, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="CyrusOne (formerly Cervalis)", value="Cervalis", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1152, 
                                 21, 
                                 y_label="Count",
                                 x_label_size = 11
                                 )

In [ ]:
iplot(chartdict, link_text="")

# Systems


In [ ]:
chartdict = chartdict = default_boxplot(chartdata, 
                            1173, 
                            7, 
                            "", 
                            "Percentage")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1180, 
                               x_label="",
                               y_label="Count",
                               title=None,
                               col_order=['Already have','1 year','2 years','>2 years','No Answer',"N/A (We Don't use Windows)",]
                                          )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = chartdict = default_boxplot(chartdata, 
                            1181, 
                            7, 
                            "", 
                            "Percentage")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1188, 
                               x_label="",
                               y_label="Count",
                               title=None,
                              col_order=['Already have','1 year','2 years','>2 years','Never','No Answer',])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = chartdict = default_boxplot(chartdata, 
                            1189, 
                            13, 
                            "", 
                            "Percentage")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="microsoft", value="Microsoft", inplace=True)
chartdata.replace(to_replace="spiceworks", value="SpiceWorks", inplace=True)
chartdata.replace(to_replace="Spiceworks", value="SpiceWorks", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1202, 
                                 13, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Kace", value="Dell Kace", inplace=True)
chartdata.replace(to_replace="DELL KACE", value="Dell Kace", inplace=True)
chartdata.replace(to_replace="secunia", value="Secunia", inplace=True)
chartdata.replace(to_replace="shavlik", value="Shavlik", inplace=True)
chartdata.replace(to_replace="SCCM (successor to wsus?)", value="SCCM", inplace=True)



chartdict = multicolumn_barchart(chartdata,
                                 1215, 
                                 9, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Proprietary built sofware", value="Proprietary", inplace=True)
chartdata.replace(to_replace="proprietary", value="Proprietary", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1224, 
                                 8, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1232, 
                                 18, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Sharepoint", value="SharePoint", inplace=True)
chartdata.replace(to_replace="MSFT Sharepoint", value="SharePoint", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1250, 
                                 15, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="LifeSize", value="Lifesize", inplace=True)
chartdata.replace(to_replace="lifesize", value="Lifesize", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1265, 
                                 10, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Commvault", value="CommVault", inplace=True)
chartdata.replace(to_replace="Asigra (managed backups)", value="Asigra", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1275, 
                                 20, 
                                 y_label="Count",
                                 x_label_size=11
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="n/a", value="None", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 1295, 
                                 11, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1306, 
                                 6, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1312, 
                                 14, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1326, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1327, 
                                 4, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 1331, y_label="Count", title="Where is your primary file storage?")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1333, 
                                 16, 
                                 y_label="Count",
                                 title="What remote file access solutions do you provide<br />(ex. iPad sync or Laptop sync)?"
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1349, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Gmail", value="Google", inplace=True)
chartdata.replace(to_replace="Abacus hosted Exchange", value="Abacus", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 1350, 
                                 9, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1359, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1360, 
                                 13, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="MediaTemple", value="Media Temple", inplace=True)
chartdata.replace(to_replace="Mediatemple ", value="Media Temple", inplace=True)
chartdata.replace(to_replace="Ovis Technologies", value="Ovis", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1373, 
                                 16, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

# Cloud

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1389, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1390, 
                                 12, 
                                 y_label="Count",
                                 othertext = False,
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
question_cols = range(1402,1409)
colnames = chartdata.columns[question_cols]
question_name = "What is your primary business driver to moving towards cloud? (Ranked)"

chartframe = chartdata[colnames]
variables = [mystr[len("What is your primary business driver to moving towards cloud? - "):] for mystr in colnames]

chartframe.columns= variables
meltframe = pd.melt(chartframe, value_vars=variables)
meltframe = meltframe.dropna(axis=0)

v1frame = meltframe.loc[meltframe["value"] == 1.0]
c1frame = pd.DataFrame(v1frame["variable"].value_counts())
c1frame.columns = ["1"]

v2frame = meltframe.loc[meltframe["value"] == 2.0]
c2frame = pd.DataFrame(v2frame["variable"].value_counts())
c2frame.columns = ["2"]
chartframe = pd.concat([c1frame, c2frame], axis=1, join_axes=[c1frame.index])    

v3frame = meltframe.loc[meltframe["value"] == 3.0]
c3frame = pd.DataFrame(v3frame["variable"].value_counts())
c3frame.columns = ["3"]
chartframe = pd.concat([chartframe, c3frame], axis=1, join_axes=[chartframe.index])    

v4frame = meltframe.loc[meltframe["value"] == 4.0]
c4frame = pd.DataFrame(v4frame["variable"].value_counts())
c4frame.columns = ["4"]
chartframe = pd.concat([chartframe, c4frame], axis=1, join_axes=[chartframe.index])    

v5frame = meltframe.loc[meltframe["value"] == 5.0]
c5frame = pd.DataFrame(v5frame["variable"].value_counts())
c5frame.columns = ["5"]
chartframe = pd.concat([chartframe, c5frame], axis=1, join_axes=[chartframe.index])    

v6frame = meltframe.loc[meltframe["value"] == 6.0]
c6frame = pd.DataFrame(v6frame["variable"].value_counts())
c6frame.columns = ["6"]
chartframe = pd.concat([chartframe, c6frame], axis=1, join_axes=[chartframe.index])    

v7frame = meltframe.loc[meltframe["value"] == 7.0]
c7frame = pd.DataFrame(v7frame["variable"].value_counts())
c7frame.columns = ["7"]
chartframe = pd.concat([chartframe, c7frame], axis=1, join_axes=[chartframe.index])    

chartframe = chartframe.fillna(value=0.0)
#v1count = v1frame.value_counts()

# sort in this order
col_order = [u'DR / BCP',
              u'IT Agility (time to mkt, portability, resource elasticity etc..)',
              u'Cost savings (initial, TCO, pay for what you use, etc..)',
              u'Innovation',
              u'Better enable mobile workforce',
              u'Move from Capex to Opex model',
              u'Critical partner forcing the move']
order_dict = dict(zip(col_order, range(len(col_order))))
chartframe["order"] = [order_dict[v] for v in chartframe.index.values]
chartframe = chartframe.sort_values("order")
del chartframe["order"]

# replace indexes like this
col_short = [u'DR / BCP',
             u'IT Agility',
             u'Cost savings',
             u'Innovation',
             u'Better enable mobile workforce',
             u'Move from Capex to Opex model',
             u'Critical partner forcing the move']
chartframe["shortindex"] = col_short
chartframe = chartframe.set_index(keys='shortindex')

chartdict = aitec_barstack(chartframe, question_name, "", "Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 1409, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1411, 
                                 8, 
                                 y_label="Count",
                                 othertext = False,
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

# Telecommunications & Network

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1419, 
                                 11, 
                                 y_label="Count",
                                 othertext = False,
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="aerohive", value="Aerohive", inplace=True)
chartdict = twocolumn_barchart(chartdata, 1430, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="gtt", value="GTT", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1432, 
                                 32, 
                                 y_label="Count",
                                 othertext = True,
                                 x_label_size=11
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1464, 
                                 18, 
                                 y_label="Count",
                                 othertext = True,
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 1482, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1484, 
                                 15, 
                                 y_label="Count",
                                 othertext = False,
                                 title='What phone system(s) does your firm use?<br />(Note: Turrets are here too, so you may have more than one)'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1499, 
                                 7, 
                                 y_label="Count",
                                 othertext = True,
                                 x_label_size=11,
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1506, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1507, 
                               x_label="",
                               y_label="Count",
                               title=None,
                              col_order=['< 3','3-10','+10','Shared function, no dedicated compliance staff','No Answer'],)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1508, 
                               x_label="",
                               y_label="Count",
                               title=None,
                               col_order=["Yes","No","No Answer"])


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 1509, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1511, 
                               x_label="",
                               y_label="Count",
                               title=None,
                               col_order=["Yes","No","No Answer","Not applicable"]                              
                              )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 1512, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1514, 
                               x_label="",
                               y_label="Count",
                               title=None,
                               col_order=["Yes","No","No Answer","Not applicable"]                                                         
                              )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 
                               1515, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 
                               1517, 
                               x_label="",
                               y_label="Count",
                               title="If you allow expert consultants, do you allow investment stafff<br />to source experts through LinkedIn or some other platform?"
                              )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1519, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 
                               1520, 
                               x_label="",
                               y_label="Count",
                               #title=""
                              )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1522, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1523, 
                               x_label="",
                               y_label="Count",
                               title="Does your firm allow the use of personal email on company's computers<br />(within or that connect to the firm's network)?",
                               col_order=['No','Yes','Yes, but monitor all attachments','Yes, but block all attachments','No Answer',]
                              )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1524, 
                                 8, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1532, 
                                 8, 
                                 y_label="Count",
#                                 col_order=['Daily','Monthly','Quarterly','Annually','Only if there is cause','Never','Random sampling','No Answer',]
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1540, 
                               x_label="",
                               y_label="Count",
                               title=None)


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1541, 
                                 9, 
                                 y_label="Count",
                                 
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 
                               1550, 
                               x_label="",
                               y_label="Count",
                               title="Does your firm use any behavioral or relationship analytics tools<br />for communication surveillance?")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1551, 
                                 8, 
                                 y_label="Count",
                                 title='If you use analytic tools for communication surveillance<br />which platform do you use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="EZE", value="Eze Castle", inplace=True)

chartdict = multicolumn_barchart(chartdata,
                                 1559, 
                                 13, 
                                 y_label="Count",
                                 title='What vendor(s) do you use for consolidated regulatory reporting<br />(e.g. Form PF, AIFMD Annex IV, etc.)?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Schwabb", value="Schwab", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 1572, 
                                 14, 
                                 y_label="Count",
                                 title='What Compliance Workflow/Personal Account Trade Monitoring<br />platforms does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1586, 
                                 14, 
                                 y_label="Count",
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1600, 
                                 12, 
                                 y_label="Count",
                                 
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="CodeRed", value="Code Red", inplace=True)
chartdata.replace(to_replace="Dynamo from Netage Solutions", value="Dynamo", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 1612, 
                                 27, 
                                 y_label="Count",
                                 title='What Client Relationship Management/<br />Investor Distribution system does your firm use?'
                                 )


In [ ]:

iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1639, 
                                 14, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = twocolumn_barchart(chartdata, 1653, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 1655, y_label="Count")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = default_boxplot(chartdata,
                                 1656, 
                                 17, 
                                 y_label="Count",
                                 title='What percent of users use each mobile provider?'
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = default_boxplot(chartdata,
                                 1673, 
                                 6, 
                                 y_label="Count",
                                 title='What percentage of mobile phone models<br />have been deployed to staff?'
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdata.replace(to_replace="Meraki", value="Cisco Meraki", inplace=True)
chartdict = multicolumn_barchart(chartdata,
                                 1679, 
                                 10, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1689, 
                                 7, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )


In [ ]:
iplot(chartdict, link_text="")

# Risk

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1696, 
                                 23, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1719, 
                                 17, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

# Research and Data Management

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1736, 
                                 14, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1750, 
                                 15, 
                                 y_label="Count",
                                 title='Which market data delivery platforms does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 1765, y_label="Count", title="Considered moving away from Bloomberg?")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 1766, y_label="Count", title="Do you have shared Bloomberg terminals?")


In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1767, 
                                 10, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1777, 
                                 13, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = onecolumn_barchart(chartdata, 1790, y_label="Count", title="Is your Research Management System cloud-based or local?")

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1791, 
                                 19, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1810, 
                                 15, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

In [ ]:
chartdict = multicolumn_barchart(chartdata,
                                 1825, 
                                 15, 
                                 y_label="Count",
                                 #title='What Client Relationship Management/Investor Distribution system does your firm use?'
                                 )

In [ ]:
iplot(chartdict, link_text="")

# That's all, folks! Many thanks for participating in the 2017 AITEC tech survey!


# The AITEC survey team

<table>
<tr><td>Druce Vertes<br />Python slide wrangler    </td><td> Ken D'Agostino<br /> Buy side wrangler </td></tr>
<tr><td>Jeff Gerson <br /> Sell side wrangler   </td><td> Hans Bleuel <br /> Fearless leader </td></tr>
<tr><td>Maura Harris <br /> Committee wrangler   </td><td> David MacPhee <br /> SurveyMonkey wrangler</td></tr>
<tr><td>Andrew Evangelidis </td><td> Zachary Frayne  </td></tr>
<tr><td>Cary Goldstein  </td><td> Jeffrey Zinkerman </td></tr>
<tr><td>Sharad Agrawal </td><td> Julio Garcia </td></tr>
<tr><td>Charles Chien</td><td> Don Choe </td></tr>
<tr><td>Conor Kiernan</td><td> Thanks to anyone I forgot</td></tr>
</table>




In [ ]:
YouTubeVideo('gBzJGckMYO4')

In [ ]:
YouTubeVideo('T1XgFsitnQw')